**NOTA IMPORTANTE**: Las preguntas están marcadas en color rojo. Para la entrega, prepara un documento separado que solo contenga las preguntas y tus respuestas.

<font color="#f00">**La entrega es un documento PDF solo con las preguntas y respuestas**</font>

Las prácticas de este curso las haremos en Python, que tiene dos posibilidades para criptografía: el paquete `PyCryptodome` y el paquete `cryptography`. Ambos son opciones válidas e intercambiables, aunque Las prácticas de este curso las haremos con `PyCryptodome`. Puedes encontrar la ayuda en: https://pycryptodome.readthedocs.io/en/latest/

Si estás siguiendo estas notas en tu PC, puedes instalarlo con: `python3 -m pip install pycryptodome`. Si las estás siguiendo en Colab, ejecuta la siguiente línea:

In [ ]:
# La siguiente línea instala CryptoDome en nuestra máquina virtual
!pip install pycryptodome
# La siguiente línea nos permitirá imprimir líneas largas
from pprint import pprint
from textwrap import wrap

# Cifrado simétrico o de clave secreta

Como hemos visto en las sesiones, e**l cifrado simétrico o de clave secreta es aquel cifrado que utiliza la misma clave para cifrar y para descifrar**. Las dos partes de la comunicación necesitan conocer la clave y mantenerla secreta para que nadie más tenga acceso a ella.

En la actualidad se usan dos cifrados simétricos ChaCha20 y AES. En estos ejercicios nos fijaremos en AES, que es el cifrado simétrico más utilizado en Internet.

## Cifrado de bloque con AES

El otro sistema de cifrado simétrico que veremos es AES.

Como ya comentamos en la parte teórica, no es suficiente con indicar que usamos AES, también es necesario especificar de qué manera, es decir, **en qué modo estamos usando AES**.

Vamos a crear:

- un `message` de 128 bits, el tamaño de bloque de AES.
- una clave `key` de 128 bits que usaremos durante todo el ejercicio.

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

# Esta es la clave secreta compartida que tienen que tener tanto el emisor como el receptor
# La clave tienen que ser 16 Bytes (128 bits) aleatorios
key = get_random_bytes(16)

# Emisor
########
# Mensaje que quiere enviar
message = b'Atacad a las dos'
print(f'Mensaje: "{message}" Tamaño={len(message) * 8} bits')
print(f'Clave: {b64encode(key)} Tamaño={len(key) * 8} bits')

### Modo ECB

El primer modo que veremos es "ECB": cada bloque se cifra de forma independiente, sin realimentación con los demás.

![center w:30em](https://github.com/Juanvvc/crypto/blob/main/ejercicios/02b/images/ECB_encryption.svg?raw=1)

Vamos a ciframos tres veces el mismo mensaje. Observa que no hay memoria, y que cifrar dos veces el mismo mensaje con la misma clave produce el mismo texto cifrado. Por eso no se debe usar nunca el modo ECB: un atacante que observa las comunicaciones quizá no sepa qué estamos diciendo, pero sí que sabe que estamos repitiendo el mensaje y eso puede ser suficiente para sus objetivos.

**Nunca debe usarse AES en modo ECB**

In [ ]:
print("Cifrado por el emisor (ECB)\n---------------------------")
cipher_emisor = AES.new(key, AES.MODE_ECB)
c1 = cipher_emisor.encrypt(message)
c2 = cipher_emisor.encrypt(message)
c3 = cipher_emisor.encrypt(message)
# El emisor envía el mismo mensaje cifrado tres veces: objserva que siempre es lo mismo
# un atacante sabe que el emisor está diciendo tres veces lo mismo ¡Eso no puede ser!
print(f"Primer mensaje cifrado c1={b64encode(c1)}")
print(f"Segundo mensaje cifrado c1={b64encode(c2)}")
print(f"Tercer mensaje cifrado c1={b64encode(c3)}")
print("\n")

# Receptor
##########
print("Descifrado por el receptor (ECB)\n--------------------------------")
cipher_receptor = AES.new(key, AES.MODE_ECB)
m1 = cipher_receptor.decrypt(c1)
m2 = cipher_receptor.decrypt(c1)
m3 = cipher_receptor.decrypt(c1)
print(f"Primero mensaje recibido={m1}")
print(f"Segundo mensaje recibido={m2}")
print(f"Tercer mensaje recibido={m3}")



### Modo CBC

Otro de los modos posible ses CBC. En CBC hay realimentación entre bloques y existe un vector de inicialización

![](https://github.com/Juanvvc/crypto/blob/main/ejercicios/02b/images/CBC_encryption.svg?raw=1)

Ciframos dos veces el mismo mensaje. Observa que tenemos que crear una variable llamada `IV` (Vector de Inicialización), y que este IV se lo tenemos que enviar al receptor. El envío del IV se hace durante el primer mensaje, antes de que el canal sea seguro, pero no hay problemas en que un atacante conozca el IV.

Fíjate: en modo CBC, aunque siempre enviamos el mismo mensaje `Atacaremos al amanecer`, **el cifrado es diferente cada vez**. Ahora un atacante no sabe que estamos enviando siempre el mismo mensaje.

In [ ]:
# Emisor
print("Cifrado por el emisor (CBC)\n---------------------------")
# Además de la clave, el emisor tiene que generar un IC
iv = get_random_bytes(16)
cipher_emisor = AES.new(key, AES.MODE_CBC, iv=iv)
c1 = cipher_emisor.encrypt(message)
c2 = cipher_emisor.encrypt(message)
c3 = cipher_emisor.encrypt(message)
print(f"Primer mensaje cifrado c1={b64encode(c1)} iv={b64encode(iv)}. El emisor tiene que enviar el IV con el primer mensaje, pero no es necesario que lo envíe también con los demás")
print(f"Segundo mensaje cifrado c1={b64encode(c2)}")
print(f"Tercer mensaje cifrado c1={b64encode(c3)}")
print("\n")

# Receptor
##########
print("Descifrado por el receptor (CBC)\n--------------------------------")
# el receptor tiene que usar la key compartida, y también el mismo IV que ha usado el emisor
cipher_receptor = AES.new(key, AES.MODE_CBC, iv=iv)
m1 = cipher_receptor.decrypt(c1)
m2 = cipher_receptor.decrypt(c2)
m3 = cipher_receptor.decrypt(c3)
print(f"Primero mensaje recibido={m1}")
print(f"Segundo mensaje recibido={m2}")
print(f"Tercer mensaje recibido={m3}")

<font color="red">
<b>PREGUNTA:</b>

1. Ahora los textos cifrados son diferentes a pesar de que estamos cifrando el mismo mensaje. ¿Por qué sucede eso?
</font>

### Padding

¿Qué pasa si tenemos que enviar mensajes más cortos que la longitud de bloque de AES? Entonces tenemos que usar algún algoritmo de padding. Es decir: marcar la longitud del mensaje original. Observa que solo podemos enviar bloques de 128 bits, y si intentamos enviar bloques más cortos o más largos saltará un error:

In [ ]:
c1 = cipher_emisor.encrypt(b'mensaje corto')

Con Cryptodome podemos usar las funciones `pad()` y `unpad()`

Observa: en este ejemplo no especificamos IV al configurar el *cipher*, pero ya sabes que en modo CBC siempre hay un IV. Si no lo especificamos, escogerá el IV al azar utilizando algoritmos segutos, y eso es muy adecuado. Tenemos que enviarle ese IV al receptor en el primer mensaje.

In [ ]:
from Crypto.Util.Padding import pad, unpad

# mensaje corto
m = b'1234'
cipher = AES.new(key, AES.MODE_CBC)
c = cipher.encrypt(pad(m, AES.block_size))
print({'iv':b64encode(cipher.iv), 'ciphertext':b64encode(c)})

Recepción:

In [ ]:
decipher = AES.new(key, AES.MODE_CBC, cipher.iv)
unpadded_m = decipher.decrypt(c)
padded_m = unpad(unpadded_m, AES.block_size)
print(f"El mensaje descifrado unpadded es r={unpadded_m} (longitud {len(unpadded_m) * 8} bits)\ny después de quitar lo que sobra m={padded_m} ")

<font color="red">
<b>PREGUNTA:</b>

1. ¿Qué sucede si se pierden mensajes en modo CBC? ¿Qué modo de AES permitiría resincronizarse?

</font>

# Cifrado asimétrico o de clave pública

En el cifrado asimétrico o de clave pública, cada participante en la comunicación tiene DOS claves: una de ellas (la clave privada) debe mantenerla totalmente secreta y no decírsela a nadie, mientras que la otra (la clave pública) puede hacerse pública.

Vamos a ver dos ejemplos de estos sistemas:

- Acuerdo de claves Diffiel-Hellman, que permite que dos personas que no se han visto nunca pueda tener una clave AES común
- RSA, que permite hacer tanto cifrado como firma electrónica

## Acuerdo de claves Diffie-Hellman

El principal problema que tiene AES y todos los sistemas de clave secreta es que la clave tiene que ser eso, secreta. ¿Cómo puede el emisor enviar la clave al receptor de forma que nadie pueda interceptarla? La respuesta es el protocolo Diffie-Hellman.

El protocolo Diffie-Hellman (D-H) permite que dos personas que no se han visto nunca acuerden una clave que después pueden utilizar para el cifrado AES o ChaCha20. Gracias a D-H, Nadie más que esas dos personas sabrán que clave han acordado, incluso si un atacante leyese todos los mensajes que se han intercambiado desde el principio. Esto se conoce como Acuerdo de Intercambio de Claves (https://en.wikipedia.org/wiki/Key_exchange)

El protocolo D-H se basa en que el problema del logaritmo discreto solo puede resolverse por fuerza bruta... a menos que ya tengas parte de la solución. Estos son los pasos que tienen que hacer Alice y Bob en D-H

1. Alice y Bob quieren comunicarse en secreto. Para ello, necesitan tener una clave secreta AES común. El primer paso es que acuerdan dos números $g$ y $p$ primos entre sí. Estos dos primos **no son secretos** y en realidad todos usamos los mismos números que se han publicado en un estándar internacional.
1. Alice escoge en secreto un número $a$. Este es un secreto que Alice no le va a decir a nadie, ni siquiera a Bob. **$a$ es la clave privada de Alice**. Bob hará lo mismo escogiendo **$b$, la clave privada de Bob.**
1. Alice y Bob calculan sus claves públicas $A$ y $B$ a partir de sus claves privadas $a$ y $b$ y los números primos $g$ y $p$ del primer paso. Estas claves públicas las puede conocer cualquiera. Alice y Bob se intercambian las claves públicas, no hace falta cifrarlas porque son públicas.
    - $Alice \rightarrow Bob: A=g^{a} \mod p$
    - $Bob \rightarrow Alice: B=g^{b} \mod p$
1. Alice, a partir de su clave privad $a$ y clave pública de Bob $B$ puede calcular un número $s$ que no envía a nadie. Bob puede hacer lo mismo a partir de su clave privada $b$ y la clave pública de Alice $A$:
    - $Alice$: $s = B^{a} \mod p = g^{ab} \mod p$
    - $Bob$: $s = A^{b} \mod p = g^{ab} \mod p$
1. Fíjate: el número $s$ calculado por Alice es el mismo que el que ha calculado Bob. Además, nadie puede calcular $s$ a partir de las claves públicas, necesitan tener las claves privadas o usar fuerza bruta. A partir de ahora, Alice y Bob puede usar $s$ como clave de cifrado en AES.  

Vamos a hacer una prueba de ejecución real con la librería https://github.com/amiralis/pyDH de Amirali Sanatinia, que es sencillo de leer y entender.

Aunque no parece haber errores evidentes, **es obligatorio utilizar librerías auditadas**. Seguiremos esta por su valor educativo, no porque sea recomendable su uso.

In [ ]:
"""
Pure Python Diffie Hellman implementation

Source: https://github.com/amiralis/pyDH

Apache License
         Version 2.0, January 2004
     Copyright 2015 Amirali Sanatinia
"""

import os
import binascii
import hashlib

# RFC 3526 - More Modular Exponential (MODP) Diffie-Hellman groups for
# Internet Key Exchange (IKE) https://tools.ietf.org/html/rfc3526

primes = {

	# 1536-bit
	5: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA237327FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 2048-bit
	14: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 3072-bit
	15: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A93AD2CAFFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 4096-bit
	16: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C934063199FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 6144-bit
	17: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C93402849236C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BDF8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1BDB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F323A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AACC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE32806A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55CDA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE12BF2D5B0B7474D6E694F91E6DCC4024FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 8192-bit
	18: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C93402849236C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BDF8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1BDB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F323A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AACC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE32806A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55CDA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE12BF2D5B0B7474D6E694F91E6DBE115974A3926F12FEE5E438777CB6A932DF8CD8BEC4D073B931BA3BC832B68D9DD300741FA7BF8AFC47ED2576F6936BA424663AAB639C5AE4F5683423B4742BF1C978238F16CBE39D652DE3FDB8BEFC848AD922222E04A4037C0713EB57A81A23F0C73473FC646CEA306B4BCBC8862F8385DDFA9D4B7FA2C087E879683303ED5BDD3A062B3CF5B3A278A66D2A13F83F44F82DDF310EE074AB6A364597E899A0255DC164F31CC50846851DF9AB48195DED7EA1B1D510BD7EE74D73FAF36BC31ECFA268359046F4EB879F924009438B481C6CD7889A002ED5EE382BC9190DA6FC026E479558E4475677E9AA9E3050E2765694DFC81F56E880B96E7160C980DD98EDD3DFFFFFFFFFFFFFFFFF,
	"generator": 2
	}
}


class DiffieHellman:
	""" Class to represent the Diffie-Hellman key exchange protocol """
	# Current minimum recommendation is 2048 bit.
	def __init__(self, group=14):
		if group in primes:
			self.p = primes[group]["prime"]
			self.g = primes[group]["generator"]
		else:
			raise Exception("Group not supported")

		self.__a = int(binascii.hexlify(os.urandom(32)), base=16)

	def get_private_key(self):
		""" Return the private key (a) """
		return self.__a

	def gen_public_key(self):
		""" Return A, A = g ^ a mod p """
		# calculate G^a mod p
		return pow(self.g, self.__a, self.p)

	def check_other_public_key(self, other_contribution):
		# check if the other public key is valid based on NIST SP800-56
		# 2 <= g^b <= p-2 and Lagrange for safe primes (g^bq)=1, q=(p-1)/2

		if 2 <= other_contribution and other_contribution <= self.p - 2:
			if pow(other_contribution, (self.p - 1) // 2, self.p) == 1:
				return True
		return False

	def gen_shared_key(self, other_contribution):
		""" Return g ^ ab mod p """
		# calculate the shared key G^ab mod p
		if self.check_other_public_key(other_contribution):
			self.shared_key = pow(other_contribution, self.__a, self.p)
			return hashlib.sha256(str(self.shared_key).encode()).digest()
		else:
			raise Exception("Bad public key from other party")


### Alice

En esta ocasión, Alice y Bob quieren acordar una clave que después usarán para las comunicaciones con AES.

Vamos a ver primero qué hace Alice.

In [ ]:
alice = DiffieHellman()
alice_pubkey = alice.gen_public_key()

print('CLAVE PRIVADA DE ALICE: a\n-------------------')
print(f'a = {alice.get_private_key()}\n')

print('\nCLAVE PÚBLICA DE ALICE: es el trío A=(g, p, g^a)\n-------------------')
print(f'g = {alice.g}')
pprint(wrap(f'p = {alice.p}'))
pprint(wrap(f'g^a = {alice_pubkey}'))

Claves de Alice:

- Clave privada es un solo número: $a_{priv}$. Este número es la clave privada de Alice y nunca saldrá del PC de Alice.
- Clave pública es un trío de números: $A_{pub}=\{g, p, g^a\}$. Esta es la clave pública de Alice, que es lo que le envía a Bob.

Nota: en realidad $g$ y $p$ no los suele escoger Alice sino que usa algún valor estándar de una tabla. Así que normalmente solo se envía $g^a$, y los valores de $g$ y $p$ son los del estándar. $g$ suele ser un primo pequeño, 2 ó 3, para acelerar los cálculos, mientras que $p$ es un número muy grande para dar seguridad al sistema. Fíjate que tanto $g$ como $p$ están simplemente en la librerías que hemos usado, no los ha escogido Alice.

### Bob

Bob calcular su par de claves públicas y privadas de una forma similar. El $g$ y $p$ que usa Bob tiene que ser el mismo que usa Alice.

In [ ]:
bob = DiffieHellman()
bob_pubkey = bob.gen_public_key()

print('CLAVE PRIVADA DE BOB: b\n-------------------')
print(f'b = {bob.get_private_key()}\n')

print('\nCLAVE PÚBLICA DE BOB: es el trío B=(g, p, g^b)\n-------------------')
print(f'g = {bob.g}')
pprint(wrap(f'p = {bob.p}'))
pprint(wrap(f'g^b = {bob_pubkey}'))

### Alice y Bob: cálculo de la clave compartida

Ahora (recuerda que todos estos cálculos son $\mod p$):

- Alice recibe de Bob $B=g^b$, y puede calcular la clave compartida usando su propia $a$: $k_{shared} = B^a=g^{ab}$
- Bob recibe de Alice $A=g^a$, y puede calcular la clave compartida usando su propia $b$: $k_{shared} = A^b=g^{ab}$
- Un espía de las comunicaciones sabe $A=g^a$ y $B=g^b$, pero no sabe $a$ ni $b$. Observa que sabiendo solo $g^a$ y $g^b$ no hay forma de calcular $g^{ab}$ y el atacante no puede calcular la clave

In [ ]:
alice_sharedkey = alice.gen_shared_key(bob_pubkey)
print(f'Clave compartida calculada por Alice: {alice_sharedkey}')
print(f'Tamaño: {len(alice_sharedkey) * 8} bits')

In [ ]:
bob_sharedkey = bob.gen_shared_key(alice_pubkey)
print(f'Clave compartida calculada por Alice: {bob_sharedkey}')
print(f'Tamaño: {len(bob_sharedkey) * 8} bits')

Y podemos comprobar que los dos tienen la misma clave compartida

In [ ]:
print(bob_sharedkey == alice_sharedkey)

<font color="red">
<b>PREGUNTAS:</b>

1. Ya tenemos una "clave compartida", pero aún hay que adaptarla para poder usarla en AES-256 o ChaCha20. ¿Cómo lo harías?
1. Los parámetros p y g de la librería son muy antiguos (RFC3526). ¿Puedes buscar otros más modernos?

</font>

## RSA

Diffie-Hellman solo puede usarse como protocolo de intercambio de clave que después se utiliza en AES.

Veremos a continuación RSA, que ya es un sistema de cifrado asimétrico completo que permite ofrecer confidencialidad y firma electrónica. Dependiendo si ciframos la clave privada o a la inversa, tenemos dos esquemas de funcionamiento:

- **Esquema de cifrado** (confidencialidad): Bob quiere enviar un mensaje a Alice. Bob cifra el mensaje con la clave pública de Bob, y solo Alice puede descifrar ese mensaje con la clave privada de Alice, que solo tiene ella
- **Esquema de firma electrónica** (autenticación): Bob quiere enviar un mensaje a Alice de forma que pueda probar que el mensaje solo lo ha podido enviar Bob. Para ello, cifra el mensaje usando la clave privada de Bob. Cualquiera puede descifrar ese mensaje usando la clave pública de Bob pero, al descifrarlo, saben que el mensaje viene necesariamente de Bob porque es el único que tiene su clave privada.

Puedes obtener confidencialidad y además autenticación ejecutando los dos esquemas a la vez: Bob cifra un mensaje con su clave privada (autenticación) y después con la clave pública de Alice (confidencialidad). De esta manera, el mensaje solo lo puede descifrar Alice y además solo lo ha podido enviar Bob.

Como comentario original, RSA también puede usarse como protocolo de acuerdo de clave, como Diffie-Hellman: una parte simplemente envía a la otra la clave AES que se va a usar, cifrada con RSA.

### Generación de claves con RSA

Vamos a generar un par de claves pública/privada con RSA de tamaño 2048 bits


In [ ]:
# Clave RSA de 2048 bits
from Crypto.PublicKey import RSA
key2048 = RSA.generate(2048)
pprint(key2048.export_key())
pprint(key2048.public_key().export_key())

<font color="red">
<b>PREGUNTAS:</b>

1. Mide cuánto tiempo necesitamos para generar una claves de un tamaño ya útil 2048.
1. Prueba a generar claves mayores, de 4096 bits y de 16384 bits, por ejemplo, que es similar en seguridad al cifrado AES de 256 bits. ¿Cuánto tiempo le lleva aproximadamente?

</font>


### Cifrado y descifrado

No es recomendable utilizar RSA "de forma pura", es decir, sin tener en cuenta muchas consideraciones sobre padding, conversiones, longitudes... que se recogen en [PKCS#1](https://en.wikipedia.org/wiki/PKCS_1). De hecho, PyCryptoDome no nos va a dejar utilizar el cifrado y descifrado directamente.

Observa que la línea siguiente resulta en un error, avisando que uses el módulo  `Crypto.Cipher.PKCS1_OAEP` en vez de las funciones directas. PKCS#1 es el estándar de cifrado de RSA y gestiona padding, randomness y otras consideraciones.

In [ ]:
key2048.encrypt(b'hola', None)

**Aunque no se debe**, para entender por qué es necesario PKCS#1 vamos a utilizar la función `_encrypt()` y `_decrypt()`, que no está documentada pero la [puedes encontrar en el código de la librería](https://github.com/Legrandin/pycryptodome/blob/4fd3013fd2fbe029a56b61fdf173ab49d3e8d179/lib/Crypto/PublicKey/RSA.py#L178)

Fíjate: vamos simplemente a cifrar el número '15' (que ocuparía 1 byte). Observa que el número '15' cifrado da un número enorme.

In [ ]:
m = 15
c = key2048._encrypt(m)
d = key2048._decrypt(c)

print(f"- Mensaje: {m}")
pprint(wrap(f"- Cifrado: {c}"))
print(f"- Descifrado: {d}")

¿Cómo podemos cifrar una cadena como 'hola'? Pues, por ejemplo, cifrando los bytes de la cadena uno a uno. Observa todo lo que ocuparía esta solución.

In [ ]:
pprint(([wrap(str(key2048._encrypt(c))) for c in b'hola']))

Otra solución es convertir la cadena a un entero con la función: `msg = int.from_bytes(b"hola mundo", "big")`

In [ ]:
msg = b"hola"
msgAsNumber = int.from_bytes(msg, "big")
print(f'Mensaje "{msg}" traducido a número: {msgAsNumber}')

ciphered = key2048._encrypt(msgAsNumber)

pprint(wrap(f'Cifrado: {key2048._encrypt(msgAsNumber)}'))

El problema de esto es que **no podemos cifrar un mensaje más grande que el tamaño en bits de la clave**, es decir, 2048 bits en este caso. Vamos a probar con "hola" * 1000, que es la cadena "hola" repetida 1000 veces

In [ ]:
key2048._encrypt(int.from_bytes(b"hola" * 1000, "big"))

Vamos a hacer las cosas bien.

Para poder usar correctamente RSA hay que seguir una serie de precauciones y procedimientos. Estas precauciones se llaman [PKCS1](https://en.wikipedia.org/wiki/PKCS_1) y cumplen una función similar a los modos de AES: usar RSA directamente es inseguro, así que hay que utilizarlo a través de PKCS#1.

Vamos a cifrar el texto `"hola"` y `"hola" * 1000` usando PKCS#1 y nuestra clave RSA que hemos generado.

In [ ]:
from Crypto.Cipher import PKCS1_OAEP

cipher = PKCS1_OAEP.new(key2048)
c = cipher.encrypt(b'hola')
pprint(wrap(f'Cifrado RSA con PKCS#1: {c}'))
print(f'Descifrado: {cipher.decrypt(c)}')

c2 = cipher.encrypt(b'hola' * 1000)
print(f'Cifrado RSA: {c2}')
pprint(wrap(f'Cifrado: {c2}'))
print(f'Cifrado: {cipher.decrypt(c2)}')



Vaya, tampoco podemos cifrar un mensaje como `"hola" * 1000` (es decir, 1000 veces la palabra "hola) ni siquiera con PKCS#1, porque el mensaje es más largo que la clave. Esta es una de las razones por las que necesitamos el cifrado híbrido, que veremos a continuación

## Cifrado híbrido

Normalmente no se cifra un mensaje en RSA, sino que se usa algún tipo de cifrado híbrido:

- Cifrar con RSA la clave AES que se usa en las comunicaciones
- Cifrar con RSA el hash de un documento para firma digital

En el tema de TLS veremos un cifrado híbrido: ciframos con RSA la clave AES que usamos para cifrar el texto.

1. Bob: Crea par de claves RSA
1. Alice: Crea clave simétrica AES. Esto es lo que se llama "clave de sesión". Alice cifra la clave AES con la clave pública de Bob. Envía mensaje
1. Alice: cifra "hola mundo" con clave AES. Envía mensaje
1. Bob: descifra clave AES con clave privada. Descifra mensaje de Alice

Entre los ejemplos de RSA precisamente verás algo así: https://pycryptodome.readthedocs.io/en/latest/src/examples.html#encrypt-data-with-rsa

<font color="red">
<b>Pregunta:</b>

1. ¿Se te ocurre por qué es necesario el cifrado híbrido y no se cifra todo con RSA y ya está?

</font>